In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import ast
import re
from collections import Counter, defaultdict

# For sparse route:
from scipy.sparse import csr_matrix, save_npz, load_npz
import joblib  # to save mappings

In [12]:
from collections import defaultdict
from tqdm import tqdm   # optional, for progress bars

tqdm.pandas()

In [2]:
df1 = pd.read_csv("order_data.csv")

In [3]:
df1.head(20)

,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_ID,ORDERS,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME
0,362204699,2156,2024-07-24,7247194287,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
1,269612955,1419,2025-02-15,791214421,"{""orders"": [{""item_details"": [{""item_name"": ""R...",Digital,WWT,ToGo
2,585330633,2249,2025-02-15,7575285208,"{""orders"": [{""item_details"": [{""item_name"": ""2...",Digital,WWT,ToGo
3,950661333,2513,2024-03-29,4253875716,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
4,434985772,1754,2024-04-08,7150407872,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
5,126084616,949,2025-02-15,3060989630,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
6,426992703,2156,2025-02-15,1655782790,"{""orders"": [{""item_details"": [{""item_name"": ""1...",Digital,WWT,ToGo
7,772568272,820,2025-02-15,4015163280,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,Delivery
8,715522716,1161,2024-06-08,3057756219,"{""orders"": [{""item_details"": [{""item_name"": ""F...",Digital,WWT,ToGo
9,591908874,820,2025-02-15,4349588916,"{""orders"": [{""item_details"": [{""item_name"": ""R...",Digital,WWT,ToGo


In [4]:
df1.describe()

,CUSTOMER_ID,STORE_NUMBER,ORDER_ID
count,1.414410e+06,1.414410e+06,1.414410e+06
mean,4.999765e+08,2.406049e+03,5.001041e+09
std,2.885254e+08,1.455188e+03,2.887713e+09
min,1.588000e+03,2.660000e+02,1.904200e+04
25%,2.503523e+08,1.161000e+03,2.498403e+09
50%,5.010379e+08,2.156000e+03,4.997326e+09
75%,7.492328e+08,4.065000e+03,7.503653e+09
max,9.999993e+08,4.970000e+03,9.999992e+09


In [25]:
len(df1["ORDER_ID"].unique())

1414410

In [5]:
df1.describe(include="object")

,ORDER_CREATED_DATE,ORDERS,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME
count,1414410,1414410,1414410,1414410,1414410
unique,446,735527,1,2,2
top,2025-03-25,"{""orders"": [{""item_details"": [{""item_name"": ""2...",Digital,WWT,ToGo
freq,11860,33654,1414410,1411089,1214921


In [6]:
df1["ORDERS"][3]

'{"orders": [{"item_details": [{"item_name": "Order Memo Item", "item_price": 0, "item_quantity": 5}, {"item_name": "Order Memo Paid", "item_price": 0, "item_quantity": 1}, {"item_name": "20 pc Grilled Wings", "item_price": 26.59, "item_quantity": 1}, {"item_name": "Order Memo ASAP", "item_price": 0, "item_quantity": 2}, {"item_name": "Ranch Dip - Regular", "item_price": 1.49, "item_quantity": 1}, {"item_name": "Order Blankline 2", "item_price": 0, "item_quantity": 1}, {"item_name": "Order Blankline 1", "item_price": 0, "item_quantity": 1}]}]}'

In [7]:
df1['ORDER_SUBCHANNEL_NAME'].unique()

array(['WWT', 'Grub Hub Marketplace'], dtype=object)

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1414410 entries, 0 to 1414409
Data columns (total 8 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   CUSTOMER_ID            1414410 non-null  int64 
 1   STORE_NUMBER           1414410 non-null  int64 
 2   ORDER_CREATED_DATE     1414410 non-null  object
 3   ORDER_ID               1414410 non-null  int64 
 4   ORDERS                 1414410 non-null  object
 5   ORDER_CHANNEL_NAME     1414410 non-null  object
 6   ORDER_SUBCHANNEL_NAME  1414410 non-null  object
 7   ORDER_OCCASION_NAME    1414410 non-null  object
dtypes: int64(3), object(5)
memory usage: 86.3+ MB


In [9]:
df1["ORDER_CREATED_DATE"] =df1["ORDER_CREATED_DATE"].astype('datetime64[ns]')

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1414410 entries, 0 to 1414409
Data columns (total 8 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   CUSTOMER_ID            1414410 non-null  int64         
 1   STORE_NUMBER           1414410 non-null  int64         
 2   ORDER_CREATED_DATE     1414410 non-null  datetime64[ns]
 3   ORDER_ID               1414410 non-null  int64         
 4   ORDERS                 1414410 non-null  object        
 5   ORDER_CHANNEL_NAME     1414410 non-null  object        
 6   ORDER_SUBCHANNEL_NAME  1414410 non-null  object        
 7   ORDER_OCCASION_NAME    1414410 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 86.3+ MB


In [11]:
df1.head()

,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_ID,ORDERS,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME
0,362204699,2156,2024-07-24,7247194287,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
1,269612955,1419,2025-02-15,791214421,"{""orders"": [{""item_details"": [{""item_name"": ""R...",Digital,WWT,ToGo
2,585330633,2249,2025-02-15,7575285208,"{""orders"": [{""item_details"": [{""item_name"": ""2...",Digital,WWT,ToGo
3,950661333,2513,2024-03-29,4253875716,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
4,434985772,1754,2024-04-08,7150407872,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo


In [26]:
# --- helper cleaners / parsers ---
def clean_item_name(name):
    """Minimal cleaning while preserving variants like '10 pc ...'"""
    if name is None:
        return None
    # ensure str, strip, collapse multiple spaces, remove leading/trailing punctuation
    s = str(name).strip()
    s = re.sub(r'\s+', ' ', s)          # collapse spaces
    s = s.strip(' \t\n\r"\'')           # trim surrounding quotes/punctuation
    return s

def parse_float_safe(x):
    """Parse price-like strings to float; return 0.0 on failure."""
    if x is None:
        return 0.0
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x)
    # remove common currency characters and commas
    s = re.sub(r'[^\d\.\-]', '', s)
    if s == '' or s == '.' or s == '-':
        return 0.0
    try:
        return float(s)
    except:
        return 0.0

def parse_qty_safe(x):
    """Parse quantity; fallback to 1 if missing/unparsable."""
    if x is None:
        return 1
    if isinstance(x, (int, float)):
        try:
            # convert floats like 2.0 to int 2
            if float(x).is_integer():
                return int(float(x))
            return float(x)
        except:
            return 1
    s = str(x)
    # extract leading numeric portion (e.g., "2", "2.0", "2 pcs")
    m = re.search(r'(\d+(\.\d+)?)', s)
    if m:
        val = float(m.group(1))
        # convert to int when integer-like
        if val.is_integer():
            return int(val)
        return val
    return 1

def extract_items_from_orders_field(orders_field):
    if pd.isna(orders_field):
        return []
    try:
        obj = json.loads(orders_field) if not isinstance(orders_field, (dict, list)) else orders_field
    except:
        try:
            obj = ast.literal_eval(orders_field)
        except:
            return []
    results = []
    if isinstance(obj, dict) and "orders" in obj:
        orders_list = obj["orders"]
    elif isinstance(obj, list):
        orders_list = obj
    else:
        return []
    seen_names = set()
    for order in orders_list:
        if isinstance(order, dict) and "item_details" in order:
            for it in order["item_details"]:
                name = clean_item_name(it.get("item_name"))
                if name and name not in seen_names:  # skip duplicates in same order
                    seen_names.add(name)
                    results.append({
                        "item_name": name,
                        "item_price": parse_float_safe(it.get("item_price")),
                        "item_quantity": parse_qty_safe(it.get("item_quantity"))
                    })
    return results

In [27]:
# ------------------------------------------------------------
# 1) Build an exploded item-level DataFrame
# ------------------------------------------------------------
# We assume df1 exists and has columns like ORDER_ID and ORDERS.
print("Parsing ORDERS and exploding into item rows (this may take some time)...")
rows = []
# If your df1 is large, you can iterate with tqdm
for idx, row in tqdm(df1.iterrows(), total=len(df1)):
    order_id = row.get('ORDER_ID')
    # parse ORDERS field
    cell = row.get('ORDERS', None)
    items_list = extract_items_from_orders_field(cell)
    for it in items_list:
        rows.append({
            'ORDER_ID': order_id,
            'item_name': it['item_name'],
            'item_price': it['item_price'],
            'item_quantity': it['item_quantity']
        })

items_df = pd.DataFrame(rows)
print("Exploded items shape:", items_df.shape)
# quick sanity
display(items_df.head())

Parsing ORDERS and exploding into item rows (this may take some time)...


100%|██████████| 1414410/1414410 [00:45<00:00, 31370.48it/s]


Exploded items shape: (5656992, 4)


,ORDER_ID,item_name,item_price,item_quantity
0,7247194287,Order Memo Not Paid,0.00,1
1,7247194287,10 pc Grilled Wings Combo,15.29,1
2,7247194287,8 pc Grilled Wings Combo,13.29,1
3,7247194287,8 pc Spicy Wings Combo,10.99,1
4,791214421,Ranch Dip - Regular,1.59,1


In [28]:
# ------------------------------------------------------------
# 2) Aggregate duplicates per (ORDER_ID, item_name)
# ------------------------------------------------------------
# Sum quantities, and compute total price contribution as sum(price * qty).
# Also keep a per-order mean unit price if price varies (for diagnostics).
agg = items_df.groupby(['ORDER_ID', 'item_name']).agg(
    item_quantity_sum = ('item_quantity', 'sum'),
    price_sum = ('item_price', lambda s: sum([parse_float_safe(x) for x in s])),  # sum of unit prices (rare)
    price_mean = ('item_price', 'mean'),
).reset_index()

# For total order price, compute sum(item_price * qty) from original items_df
items_df['price_times_qty'] = items_df['item_price'].astype(float) * items_df['item_quantity'].astype(float)
total_price_by_order = items_df.groupby('ORDER_ID', sort=False)['price_times_qty'].sum().reset_index().rename(columns={'price_times_qty':'total_order_price'})

# rename aggregated quantity to 'qty' to pivot
agg = agg.rename(columns={'item_quantity_sum':'item_quantity'})

In [29]:
# ------------------------------------------------------------
# 3) Filter item set to only items with non-zero price anywhere (your request)
# ------------------------------------------------------------
items_with_nonzero_price = items_df.loc[items_df['item_price'] > 0, 'item_name'].unique().tolist()
print(f"Unique items with price > 0: {len(items_with_nonzero_price)}")

# If you want to limit to top-K frequent items (optional), do it here:
# top_k = 200
# top_items = items_df.groupby('item_name')['item_quantity'].sum().nlargest(top_k).index.tolist()
# items_to_keep = [i for i in items_with_nonzero_price if i in top_items]

items_to_keep = items_with_nonzero_price  # as per your request

# Keep only rows for these items
agg_filtered = agg[agg['item_name'].isin(items_to_keep)].copy()

Unique items with price > 0: 130


In [30]:
# ------------------------------------------------------------
# 4) Pivot to get one column per item, value = quantity (0 if not present)
# ------------------------------------------------------------
pivot = agg_filtered.pivot_table(
    index='ORDER_ID',
    columns='item_name',
    values='item_quantity',
    aggfunc='sum',
    fill_value=0
)

# Optionally, ensure column ordering is stable (sorted or leave as-is)
pivot = pivot.reindex(sorted(pivot.columns), axis=1)

print("Pivot shape (orders x items):", pivot.shape)
# convert quantities to integers where applicable
pivot = pivot.fillna(0)
# if all values are whole numbers, cast to int
if (pivot % 1 == 0).all().all():
    pivot = pivot.astype(int)

Pivot shape (orders x items): (1414410, 130)


In [31]:
# ------------------------------------------------------------
# 5) Merge pivot & total_order_price back into df1
# ------------------------------------------------------------
# make sure ORDER_ID in df1 is the same dtype as pivot index
df1_index_type = df1['ORDER_ID'].dtype
pivot = pivot.reset_index()

df_merged = df1.merge(pivot, on='ORDER_ID', how='left')
# fill missing item columns with 0 (orders with no items kept)
item_cols = [c for c in pivot.columns if c != 'ORDER_ID']
df_merged[item_cols] = df_merged[item_cols].fillna(0)

# add total_order_price (0 if missing)
df_merged = df_merged.merge(total_price_by_order, on='ORDER_ID', how='left')
df_merged['total_order_price'] = df_merged['total_order_price'].fillna(0.0)

print("Final merged shape:", df_merged.shape)
display(df_merged.head())

Final merged shape: (1414410, 139)


,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_ID,ORDERS,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME,$19.99 Crispy Feast,10 pc Grilled Wings,...,Sports Drink,Sub Box,Triple Chocolate Cake,Triple Feast Deal,Veggie Sticks,Veggie Sticks Spicy,Voodoo Fries - Large,Voodoo Fries - Regular,Wings Lunch Combo,total_order_price
0,362204699,2156,2024-07-24,7247194287,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo,0,0,...,0,0,0,0,0,0,0,0,0,39.57
1,269612955,1419,2025-02-15,791214421,"{""orders"": [{""item_details"": [{""item_name"": ""R...",Digital,WWT,ToGo,0,0,...,0,0,0,0,0,0,0,0,0,70.57
2,585330633,2249,2025-02-15,7575285208,"{""orders"": [{""item_details"": [{""item_name"": ""2...",Digital,WWT,ToGo,0,0,...,0,0,0,0,0,0,0,0,0,16.99
3,950661333,2513,2024-03-29,4253875716,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo,0,0,...,0,0,0,0,0,0,0,0,0,28.08
4,434985772,1754,2024-04-08,7150407872,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo,0,0,...,0,0,0,0,0,0,0,0,0,24.58


In [32]:
df_testing_v3 = df_merged.head(50)

In [33]:
df_testing_v3.to_csv("df_testing_v3.csv")

In [34]:
# Drop the ORDERS column
df_merged = df_merged.drop(columns=['ORDERS'])

# Round total_order_price to 2 decimal places
df_merged['total_order_price'] = df_merged['total_order_price'].round(2)

# Save to CSV
output_path = "order_data_cleaned_and_encoded.csv"
df_merged.to_csv(output_path, index=False)

print(f"File saved to {output_path}")

File saved to order_data_cleaned_and_encoded.csv
